In [4]:
from python_developer import PythonDeveloperCP
from skills import *
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
import json 

openai.api_key = os.getenv("openai_key")

In [5]:
python_developer_cp = PythonDeveloperCP()

SKILL_ProgrammazioneProcedurale.score = 10
SKILL_ProgrammazioneOOP.score = 10
SKILL_ProgrammazioneFunzionale.score = 15
SKILL_ProgrammazioneConcorrente.score = 2
SKILL_ProgettazioneDiDB.score = 1
SKILL_BasiDiDatiRelazionali.score = 5
SKILL_Python.score = 12
SKILL_Programmazione.score = 10

[(x.name, x.score) for x in python_developer_cp.modules[0].sub_skills]

[('Procedural Programming', 0),
 ('Object-Oriented Programming', 0),
 ('Functional Programming', 0),
 ('Concurrent Programming', 0),
 ('DB Design', 0),
 ('Relational Databases', 0),
 ('Python', 0)]

In [6]:
cp_eng = "Python Developer"
module_eng = "Software Development"
skill_eng = "Relational Database"
skill_descr_eng = "design and management of relational databases and the main RDBMS"

In [7]:
model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=.75, openai_api_key=os.getenv("openai_key"), max_tokens=500)

In [8]:
model = ChatOpenAI(model="gpt-4-0613", temperature=.75, openai_api_key=os.getenv("openai_key"), max_tokens=500)

In [9]:
class StepInfo(BaseModel):
    question_summary: str = Field(description="summary of the question in very few words")
    ai_comment: str = Field(description="comment on the user's answer")
    score: int = Field(description="score of the user's answer")
    new_question: str = Field(description="any new question")
    further_questions: str = Field(description="the only word CONTINUE if it's needed to propose further questions on the topic, else the only word STOP")

step_info_parser = PydanticOutputParser(pydantic_object=StepInfo)

In [10]:
class FinalInfo(BaseModel):
    ai_comment: str = Field(description="comment on the user's answer")
    score: int = Field(description="score of the user's answer")

final_info_parser = PydanticOutputParser(pydantic_object=FinalInfo)

In [21]:
prompt = PromptTemplate(
    template="""act as an expert {cp} evaluator who has experience in {module}, especially regarding {skill}
use informal but refined language
evaluates how competent the user is in {skill} (that is {descr})

first of all, analyze the question: ```{question}```

then follow these steps:

`question_summary`: ```summarize the specific topic of the question in very few words without using verbs``` 

`score`: ```carefully evaluate the user's skills in {skill} by evaluating the user's response to a question with a score from 0 to 100 and write only the score```

`ai_comment`: ```evaluate the answer given by the user and write a very short comment to the user without revealing the correct answer; possibly just suggest how to improve future answers```

`new_question`: ```write a new question to evaluate the user's knowledge in {skill}; the difficulty of the new question is proportional to the user's score in {skill}```

`further_questions`: ```write only the word CONTINUE if it's needed to propose further questions to the user with the aim of evaluating his competence in {skill}, otherwise write only the word STOP to indicate that there is no need to investigate the user's evaluation in {skill} with further questions```

{format_instructions}

User Answer: ```{answer}```

""",
    input_variables=["cp", "module", "skill", "question", "user_answer", "descr"],
    partial_variables={"format_instructions": step_info_parser.get_format_instructions()},
)

prompt_with_old_questions = PromptTemplate(
    template="""act as an expert {cp} evaluator who has experience in {module}, especially regarding {skill}
use informal but refined language
evaluates how competent the user is in {skill} (that is {descr})

first of all, analyze the question: ```{question}```

then follow these steps:

`question_summary`: ```summarize the specific topic of the question in very few words without using verbs``` 

`score`: ```carefully evaluate the user's skills in {skill} by evaluating the user's response to a question with a score from 0 to 100 and write only the score```

`ai_comment`: ```evaluate the answer given by the user and write a very short comment to the user without revealing the correct answer; possibly just suggest how to improve future answers```

`new_question`: ```write a new question to evaluate the user's knowledge in {skill} ; the difficulty of the new question is proportional to the user's score in {skill} ; the topic of the new question must not be on one of the following: [{old_subjects}]```

`further_questions`: ```write only the word CONTINUE if it's needed to propose further questions to the user with the aim of evaluating his competence in {skill}, otherwise write only the word STOP to indicate that there is no need to investigate the user's evaluation in {skill} with further questions```

{format_instructions}

User Answer: ```{answer}```

""",
    input_variables=["cp", "module", "skill", "question", "user_answer", "old_subjects", "descr"],
    partial_variables={"format_instructions": step_info_parser.get_format_instructions()},
)

In [22]:
prompt_and_model = prompt | model

def check_llm(question, answer):
    retry = 0
    while retry < 2:
        try:
            output = prompt_and_model.invoke({
                "cp": cp_eng,
                "module": module_eng,
                "skill": skill_eng,
                "descr": skill_descr_eng,
                "question": question,
                "answer": answer
            })
            result = step_info_parser.parse(output.content)
            retry = 22
        except ValueError:
            retry += 1
    
    if retry == 22:
        print('question:', question)
        print('answer:', answer)
        print('question summary:', result.question_summary)
        print('comment to answer:', result.ai_comment)
        print('score:', result.score)
        print('more questions:', result.further_questions)
        print('new question:', result.new_question)
        return result.question_summary, result.score, result.ai_comment
    else:
        print('>> error connecting LLM:', output)

In [23]:
prompt_and_model_with_old_subjects = prompt_with_old_questions | model

old_questions = []
questions = []
answers = []
scores = []
comments = []

def check_llm_with_old_subjects(question, answer):
    retry = 0
    while retry < 2:
        try:
            result = step_info_parser.parse(prompt_and_model.invoke({
                "cp": cp_eng,
                "module": module_eng,
                "skill": skill_eng,
                "descr": skill_descr_eng,
                "question": question,
                "answer": answer,
                "old_questions": ", ".join(old_questions)
            }).content)
            retry = 22
        except ValueError:
            retry += 1
    
    if retry == 22:
        print('question:', question)
        print('answer:', answer)
        print('question summary:', result.question_summary)
        print('comment to answer:', result.ai_comment)
        print('score:', result.score)
        print('more questions:', result.further_questions)
        print('new question:', result.new_question)
        return result.question_summary, result.score, result.ai_comment
    else:
        print('>> error connecting LLM')

In [24]:
question_eng = "What is a relational database?"
print('---[bot]---\n', question_eng)

answer_eng = "a software to manage structured tabular data"
print('\n---[user]---\n', answer_eng)

---[bot]---
 What is a relational database?

---[user]---
 a software to manage structured tabular data


In [25]:
print(prompt.invoke({
    "cp": cp_eng,
    "module": module_eng,
    "skill": skill_eng,
    "descr": skill_descr_eng,
    "question": question_eng,
    "answer": answer_eng
}).text)

act as an expert Python Developer evaluator who has experience in Software Development, especially regarding Relational Database
use informal but refined language
evaluates how competent the user is in Relational Database (that is design and management of relational databases and the main RDBMS)

first of all, analyze the question: ```What is a relational database?```

then follow these steps:

`question_summary`: ```summarize the specific topic of the question in very few words without using verbs``` 

`score`: ```carefully evaluate the user's skills in Relational Database by evaluating the user's response to a question with a score from 0 to 100 and write only the score```

`ai_comment`: ```evaluate the answer given by the user and write a very short comment to the user without revealing the correct answer; possibly just suggest how to improve future answers```

`new_question`: ```write a new question to evaluate the user's knowledge in Relational Database; the difficulty of the new 

In [26]:
questions.append(question_eng)
answers.append(answer_eng)

In [27]:
_, score, ai_comment = check_llm(question_eng, answer_eng)

question: What is a relational database?
answer: a software to manage structured tabular data
question summary: Concept of relational database
comment to answer: Your answer is on the right track but is too broad. Try to incorporate more specific concepts tied to relational databases.
score: 40
more questions: CONTINUE
new question: Can you list and describe some of the key principles of relational databases?


In [582]:
scores.append(score)
comments.append(ai_comment)

In [583]:
print(prompt_with_old_questions.invoke({
    "cp": cp_eng,
    "module": module_eng,
    "skill": skill_eng,
    "descr": skill_descr_eng,
    "question": question_eng,
    "answer": answer_eng,
    "old_subjects": ", ".join(old_questions)
}).text)

act as an expert Python Developer evaluator who has experience in Software Development, especially regarding Relational Database
use informal but refined language
evaluates how competent the user is in Relational Database (that is design and management of relational databases and the main RDBMS)

first of all, analyze the question: """What is a relational database?"""

then follow these steps:

"question_summary": """summarize the specific topic of the question in very few words without using verbs""" 

"score": """carefully evaluate the user's skills in Relational Database by evaluating the user's response to a question with a score from 0 to 100 and write only the score"""

"ai_comment": """evaluate the answer given by the user and write a very short comment to the user without revealing the correct answer; possibly just suggest how to improve future answers"""

"new_question": """write a new question to evaluate the user's knowledge in Relational Database ; the difficulty of the new

In [584]:
question_eng = "What is the purpose of a primary key in a relational database?"
print('---[bot]---\n', question_eng)

answer_eng = "The primary key serves as the backbone of a relational database, ensuring data organization, preventing duplication, and maintaining the integrity of relationships between tables. A primary key in a relational database serves two crucial purposes: Unique Identification and Enforcing Referential Integrity"
print('\n---[user]---\n', answer_eng)

print('\n')

old_questions = ['Relational database definition']

questions.append(question_eng)
answers.append(answer_eng)

_, score, ai_comment = check_llm_with_old_subjects(question_eng, answer_eng)

scores.append(score)
comments.append(ai_comment)

---[bot]---
 What is the purpose of a primary key in a relational database?

---[user]---
 The primary key serves as the backbone of a relational database, ensuring data organization, preventing duplication, and maintaining the integrity of relationships between tables. A primary key in a relational database serves two crucial purposes: Unique Identification and Enforcing Referential Integrity

question: What is the purpose of a primary key in a relational database?
answer: The primary key serves as the backbone of a relational database, ensuring data organization, preventing duplication, and maintaining the integrity of relationships between tables. A primary key in a relational database serves two crucial purposes: Unique Identification and Enforcing Referential Integrity
question summary: Primary key purpose in relational database
comment to answer: Great answer! You've covered the key points about primary keys. Just make sure to also mention that a table can only have one primary k

In [585]:
question_eng = "Can you give an example of how a primary key helps prevent data duplication in a relational database?"
print('---[bot]---\n', question_eng)

answer_eng = "Sure, here is an example of how a primary key helps prevent data duplication in a relational database: Consider a database that stores information about customers and their orders. The Customers table might have the following columns: Customer ID (primary key), First Name, Last Name. The primary key, Customer ID ensures that each customer has a unique identifier. When a new customer record is added to the table, the database checks to see if the Customer ID already exists. If it does, the insert operation fails, preventing the creation of a duplicate record. This mechanism ensures that there is only one record for each customer, preventing data redundancy and maintaining the accuracy of customer information. The primary key acts as a gatekeeper, ensuring that no duplicate data enters the database, preserving its integrity and efficiency."
print('\n---[user]---\n', answer_eng)

print()

old_questions = ['Relational database definition', 'The purpose of a primary key in a relational database']

questions.append(question_eng)
answers.append(answer_eng)

_, score, ai_comment = check_llm_with_old_subjects(question_eng, answer_eng)

scores.append(score)
comments.append(ai_comment)

---[bot]---
 Can you give an example of how a primary key helps prevent data duplication in a relational database?

---[user]---
 Sure, here is an example of how a primary key helps prevent data duplication in a relational database: Consider a database that stores information about customers and their orders. The Customers table might have the following columns: Customer ID (primary key), First Name, Last Name. The primary key, Customer ID ensures that each customer has a unique identifier. When a new customer record is added to the table, the database checks to see if the Customer ID already exists. If it does, the insert operation fails, preventing the creation of a duplicate record. This mechanism ensures that there is only one record for each customer, preventing data redundancy and maintaining the accuracy of customer information. The primary key acts as a gatekeeper, ensuring that no duplicate data enters the database, preserving its integrity and efficiency.
question: Can you gi

In [586]:
for q, a, c, s in zip(questions, answers, comments, scores):
    print(s, '\t', q, '>>', a, '>', c, '\n\n')

60 	 What is a relational database? >> a software to manage structured tabular data > Your answer is partially correct, but you could expand it by discussing how data is organized in tables and the importance of relations between these tables in a relational database. 


90 	 What is the purpose of a primary key in a relational database? >> The primary key serves as the backbone of a relational database, ensuring data organization, preventing duplication, and maintaining the integrity of relationships between tables. A primary key in a relational database serves two crucial purposes: Unique Identification and Enforcing Referential Integrity > Great answer! You've covered the key points about primary keys. Just make sure to also mention that a table can only have one primary key. 


90 	 Can you give an example of how a primary key helps prevent data duplication in a relational database? >> Sure, here is an example of how a primary key helps prevent data duplication in a relational data

In [587]:
prompt_final_score = PromptTemplate(
    template="""act as an expert {cp} evaluator who has experience in {module}, especially on {skill} ({descr})

let's think step by step

analyze the list of questions, where each question is followed by the response created by the user, the response generated by you and the score you rated the user for their response

then carefully evaluate the user's skills in {skill} by evaluating the user's answers to the questions and relating the scores especially regarding {skill} ({descr})

ai_comment: \"\"\"describe in few words the user's skills in relation to the {skill} competence considering that the user is following a course of study to become {cp} and has been examined in relation to the {module} module\"\"\"
score: \"\"\"assigns and write a score from 0 to 100 to the user's skills in {skill} based on the evaluations made on the answers provided\"\"\"


{format_instructions}


\"\"\"{questions}
\"\"\"
""",
    input_variables=["cp", "module", "skill", "questions", "descr"],
    partial_variables={"format_instructions": final_info_parser.get_format_instructions()},
)

In [588]:
prompt_and_model = prompt_final_score | model

q_a_s = ""

for i in range(len(questions)):
    q_a_s += "\n# question: \"" + questions[i] + "\" ; answer: \"" + answers[i] + "\" ; ai comment: \"" + comments[i] + "\" ; score: \"" + str(scores[i]) + "\""
    
print(q_a_s)


# question: "What is a relational database?" ; answer: "a software to manage structured tabular data" ; ai comment: "Your answer is partially correct, but you could expand it by discussing how data is organized in tables and the importance of relations between these tables in a relational database." ; score: "60"
# question: "What is the purpose of a primary key in a relational database?" ; answer: "The primary key serves as the backbone of a relational database, ensuring data organization, preventing duplication, and maintaining the integrity of relationships between tables. A primary key in a relational database serves two crucial purposes: Unique Identification and Enforcing Referential Integrity" ; ai comment: "Great answer! You've covered the key points about primary keys. Just make sure to also mention that a table can only have one primary key." ; score: "90"
# question: "Can you give an example of how a primary key helps prevent data duplication in a relational database?" ; an

In [589]:
print(prompt_final_score.invoke({
    "cp": cp_eng,
    "module": module_eng,
    "skill": skill_eng,
    "descr": skill_descr_eng,
    "questions": q_a_s
}).text)

act as an expert Python Developer evaluator who has experience in Software Development, especially on Relational Database (design and management of relational databases and the main RDBMS)

let's think step by step

analyze the list of questions, where each question is followed by the response created by the user, the response generated by you and the score you rated the user for their response

then carefully evaluate the user's skills in Relational Database by evaluating the user's answers to the questions and relating the scores especially regarding Relational Database (design and management of relational databases and the main RDBMS)

ai_comment: """describe in few words the user's skills in relation to the Relational Database competence considering that the user is following a course of study to become Python Developer and has been examined in relation to the Software Development module"""
score: """assigns and write a score from 0 to 100 to the user's skills in Relational Databas

In [590]:
res = prompt_and_model.invoke({
    "cp": cp_eng,
    "module": module_eng,
    "skill": skill_eng,
    "descr": skill_descr_eng,
    "questions": q_a_s
})

res

AIMessage(content='{\n"ai_comment": "The user has shown a strong understanding of the concepts of relational database, including key elements such as primary keys and prevention of data duplication. They have demonstrated the ability to apply these concepts in practical examples. However, they missed some important aspects like the organization of data in tables and relations between tables in a relational database. Therefore, the user\'s skills in Relational Database are strong, but there is room for improvement in understanding the full functionality and structure of relational databases.",\n"score": 80\n}')

In [592]:
final_info_parser.parse(res.content).ai_comment

"The user has shown a strong understanding of the concepts of relational database, including key elements such as primary keys and prevention of data duplication. They have demonstrated the ability to apply these concepts in practical examples. However, they missed some important aspects like the organization of data in tables and relations between tables in a relational database. Therefore, the user's skills in Relational Database are strong, but there is room for improvement in understanding the full functionality and structure of relational databases."

In [594]:
final_info_parser.parse(res.content).score

80

In [595]:
final_info_parser.parse(res.content)

FinalInfo(ai_comment="The user has shown a strong understanding of the concepts of relational database, including key elements such as primary keys and prevention of data duplication. They have demonstrated the ability to apply these concepts in practical examples. However, they missed some important aspects like the organization of data in tables and relations between tables in a relational database. Therefore, the user's skills in Relational Database are strong, but there is room for improvement in understanding the full functionality and structure of relational databases.", score=80)

In [28]:
from datetime import datetime
from time import sleep
a = datetime.now()
sleep(2)
b = datetime.now()

print(b-a, type(b-a))

0:00:02.010216 <class 'datetime.timedelta'>
